In [ ]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai langchain-groq


In [2]:
import os

# os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API'

```
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine
```

In [5]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

planner = (
    ChatPromptTemplate.from_template("Gere um argumento sobre: {input}")
    | ChatGroq(model_name="mixtral-8x7b-32768")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "Liste os prós ou aspectos positivos de {base_response}"
    )
    | ChatGroq(model_name="mixtral-8x7b-32768")
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "Liste os contras ou aspectos negativos de {base_response}"
    )
    | ChatGroq(model_name="mixtral-8x7b-32768")
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Gere uma resposta final dada a crítica"),
        ]
    )
    | ChatGroq(model_name="mixtral-8x7b-32768")
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [ ]:
chain.invoke({"input": "scrum"})